In [2]:
import cv2

In [22]:

# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# from keras.layers import Input, Dense, Reshape, Flatten, Dropout
# from keras.layers import BatchNormalization, Activation, ZeroPadding2D
# from keras.layers.advanced_activations import LeakyReLU
# from keras.layers.convolutional import UpSampling2D, Conv2D
# from keras.models import Sequential, Model
# #from keras.optimizers import Adam
# from keras.utils import np_utils

# from keras.backend import tensorflow_backend

import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
# import rarfile as rar
from pathlib import Path

np.random.seed(0)
np.random.RandomState(0)
tf.random.set_seed(0)

config = tf.compat.v1.ConfigProto(gpu_options=tf.compat.v1.GPUOptions(allow_growth=True))
# session = tf.compat.v1.Session(config=config)
# tf.compat.v1.keras.backend.set_session(session)

root_dir = str(Path('../img').resolve())
class_names = os.listdir(root_dir)
print(class_names)


['Black']


In [15]:
# import keras
print(keras.__version__) 

2.5.0


In [36]:
class DCGAN():
    
    def __init__(self):

        self.class_names = os.listdir(root_dir)

        self.shape = (8, 8, 1) #(8, 8, 3)
        self.z_dim = 100

        optimizer = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)

        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        ori_noise, self.generator = self.build_generator()
        # self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        
        z = tf.keras.Input(shape=(self.z_dim,))
        img = self.generator(z)

        elems = img
        
        
        print("imgs shape (before mapping): " + str(elems.shape))
        #op = tf.map_fn(self.wavelet, elems)
#         result = sess.run(op)

#         for elem in elems:
#             print('hoge', type(elem))

#         img = self.wavelet(img)

        self.discriminator.trainable = False

        valid = self.discriminator(img)

        self.combined = keras.Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    # must under eagar mode
    def tensor_to_array(self, tensor1):
        return np.array(tensor1)


    def build_generator(self):
        noise_shape = (self.z_dim,)

        model = tf.keras.Sequential()

        model.add(layers.Dense(128 * 8 * 8, activation="relu", input_shape=noise_shape))
        model.add(layers.Reshape((8, 8, 128)))
        model.add(layers.BatchNormalization(momentum=0.8))
        #model.add(UpSampling2D())
        model.add(layers.Conv2D(128, kernel_size=3, padding="same"))
        model.add(layers.Activation("relu"))
        model.add(layers.BatchNormalization(momentum=0.8))
        #model.add(UpSampling2D())
        model.add(layers.Conv2D(64, kernel_size=3, padding="same"))
        model.add(layers.Activation("relu"))
        model.add(layers.BatchNormalization(momentum=0.8))
        model.add(layers.Conv2D(1, kernel_size=3, padding="same"))
        model.add(layers.Activation("tanh"))

        #model.summary()

        noise = keras.Input(shape=noise_shape)
        img = model(noise)
        print("(in generator)", img.shape)

        return noise, keras.Model(noise, img)

    def build_discriminator(self):
        img_shape = self.shape

        #4 機械学習（人工知能）モデルの作成
        #2021/6/2「Conv2D」を使ってディープラーニング技術でおなじみの「畳み込みニューラルネットワーク」（CNN）を実装
        model = tf.keras.Sequential()
        
        model = keras.Sequential()
        model.add(layers.Conv2D(48, (2, 2), padding='same', input_shape=(8, 8, 1), activation='relu'))
        model.add(layers.Conv2D(128, (2, 2), padding='same', activation='relu'))
        model.add(layers.Dropout(0.25))
        model.add(layers.Flatten())
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(1, activation='sigmoid'))

        model.summary()
            
        img = keras.Input(shape=img_shape)
        validity = model(img)
        
        #model.summary()

        return keras.Model(img, validity)
    
    def build_discriminator2(self):
        
        img_shape = self.shape
        
        model = Sequential()
        
        model.add(Conv2D(48, (1, 1), padding='same', input_shape=(4, 4, 3), activation='relu'))
        model.add(Conv2D(128, (2, 2), padding='same', activation='relu'))
        model.add(Dropout(0.5))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        
        model.summary()
        
        img = Input(shape=img_shape)
        
#         w_image = np.zeros((4, 4, 3))
#         for num in range(0, 4):
#             for ber in range(0, 4):
#                 w_image[num][ber][0] = (img[num*2][ber*2] - img[num*2+1][ber*2] + img[num*2][ber*2+1] - img[num*2+1][ber*2+1]) / 4
#                 w_image[num][ber][1] = (img[num*2][ber*2] + img[num*2+1][ber*2] - img[num*2][ber*2+1] - img[num*2+1][ber*2+1]) / 4
#                 w_image[num][ber][2] = (img[num*2][ber*2] - img[num*2+1][ber*2] - img[num*2][ber*2+1] + img[num*2+1][ber*2+1]) / 4
        #w_images.append(w_image)
        validity = model(img)
        

        return Model(img, validity)

    def build_combined(self):
        self.discriminator.trainable = False
        model = Sequential([self.generator, self.discriminator])

        return model

    def train(self, iterations, batch_size=128, save_interval=50, model_interval=1000, check_noise=None, r=5, c=5):

        X_train, labels = self.load_imgs()

        half_batch = int(batch_size / 2)

        X_train = (X_train.astype(np.float32) - 127.5) / 127.5

        for iteration in range(iterations):

            # ------------------
            # Training Discriminator
            # -----------------
            idx = np.random.randint(0, X_train.shape[0], half_batch)

            imgs = X_train[idx]

            noise = np.random.uniform(-1, 1, (half_batch, self.z_dim))

            gen_imgs = self.generator.predict(noise)
            print("gen_predict imgs", type(gen_imgs))

            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))

            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # -----------------
            # Training Generator
            # -----------------

            noise = np.random.uniform(-1, 1, (batch_size, self.z_dim))

            g_loss = self.combined.train_on_batch(noise, np.ones((batch_size, 1)))

            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (iteration, d_loss[0], 100 * d_loss[1], g_loss))

            model_dir = Path('ganmodels')
            model_dir.mkdir(exist_ok=True)
            if iteration % save_interval == 0:
                self.save_imgs(iteration, check_noise, r, c)
                start = np.expand_dims(check_noise[0], axis=0)
                end = np.expand_dims(check_noise[1], axis=0)
                resultImage = self.visualizeInterpolation(start=start, end=end)
                cv2.imwrite("images/latent/" + "latent_{}.png".format(iteration), resultImage)
                if iteration % model_interval == 0:
                    self.generator.save(str(model_dir)+"/dcgan-{}-iter.h5".format(iteration))

    def save_imgs(self, iteration, check_noise, r, c):
        noise = check_noise
        gen_imgs = self.generator.predict(noise)

        # 0-1 rescale
        gen_imgs = 0.5 * gen_imgs + 0.5
        #print(type(gen_imgs)
        gen_imgs = np.squeeze(gen_imgs)
        print(gen_imgs.shape)
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :])#, :])
                axs[i, j].axis('off')
                cnt += 1
        fig.savefig('images/gen_imgs/watermark_%d.png' % iteration)

        plt.close()
        
    def get_class_one_hot(self, class_str):
        label_encoded = self.class_names.index(class_str)

        label_hot = keras.utils.to_categorical(label_encoded, len(self.class_names))
        label_hot = label_hot

        return label_hot

    def load_imgs(self, ret_wavelet=False):

#         img_paths = []
#         labels = []
#         images = []
#         for cl_name in self.class_names:
#             img_names = os.listdir(os.path.join(root_dir, cl_name))
#             for img_name in img_names:
#                 img_paths.append(os.path.abspath(os.path.join(root_dir, cl_name, img_name)))
#                 hot_cl_name = self.get_class_one_hot(cl_name)
#                 labels.append(hot_cl_name)

#         for img_path in img_paths:
#             img = cv2.imread(img_path)
#             img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#             images.append(img)

#         images = np.array(images)

#        return (np.array(images), np.array(labels))
        img_paths = []
        labels = []
        images = []
        w_images = []

        for cl_name in class_names:
            print(cl_name)
            img_dir_names = os.listdir(os.path.join(root_dir, cl_name))

            for img_dir_name in img_dir_names:
                img_names = os.listdir(os.path.join(root_dir, cl_name, img_dir_name))

                for img_name in img_names:
                    img_paths.append(os.path.abspath(os.path.join(root_dir, cl_name, img_dir_name, img_name)))
                    hot_cl_name = self.get_class_one_hot(cl_name)
                    labels.append(hot_cl_name)

        for img_path in img_paths:
            img = cv2.imread(img_path)

            # (8, 8, 3) -> (8, 8, 1)へ次元を削減 すべて同じ値なので
            img = np.delete(img, [1, 2], 2)
            images.append(img)
            w_image = np.zeros((4, 4, 3))
            MM_image = np.zeros((4,4,2))
            for num in range(0, 4):
                for ber in range(0, 4):
                    w_image[num][ber][0] = (img[num*2][ber*2] - img[num*2+1][ber*2] + img[num*2][ber*2+1] - img[num*2+1][ber*2+1]) / 4
                    w_image[num][ber][1] = (img[num*2][ber*2] + img[num*2+1][ber*2] - img[num*2][ber*2+1] - img[num*2+1][ber*2+1]) / 4
                    w_image[num][ber][2] = (img[num*2][ber*2] - img[num*2+1][ber*2] - img[num*2][ber*2+1] + img[num*2+1][ber*2+1]) / 4
                    MM_image[num][ber][0] = max([w_image[num][ber][0],w_image[num][ber][1],w_image[num][ber][2]])
                    MM_image[num][ber][1] = min([w_image[num][ber][0],w_image[num][ber][1],w_image[num][ber][2]])
            #w_images.append(MM_image)
            w_images.append(w_image)

        images = np.array(images)

        if ret_wavelet:
            return np.array(w_images), np.array(labels)
        else:
            return np.array(images), np.array(labels)

   

    def visualizeInterpolation(self, start, end, save=True, nbSteps=10):
        print("Generating interpolations...")

        steps = nbSteps
        latentStart = start
        latentEnd = end

        startImg = self.generator.predict(latentStart)
        endImg = self.generator.predict(latentEnd)

        vectors = []

        alphaValues = np.linspace(0, 1, steps)
        for alpha in alphaValues:
            vector = latentStart * (1 - alpha) + latentEnd * alpha
            vectors.append(vector)

        vectors = np.array(vectors)

        resultLatent = None
        resultImage = None

        for i, vec in enumerate(vectors):
            gen_img = np.squeeze(self.generator.predict(vec), axis=0)
            gen_img = (0.5 * gen_img + 0.5) * 255
            interpolatedImage = gen_img #cv2.cvtColor(gen_img, cv2.COLOR_RGB2BGR)
            interpolatedImage = interpolatedImage.astype(np.uint8)
            resultImage = interpolatedImage if resultImage is None else np.hstack([resultImage, interpolatedImage])

        return resultImage
    
    def wavelet(self, img):
        #images.append(img)

        print(img)
        w_images = []
        print("img shape (after mapping):", img.shape)
        
        
#         w_image = tf.Variable(tf.zeros((4,4,3)))
        #w_image = tf.zeros((4,4,3))
        wave = tf.Variable(lambda: tf.zeros([4,4,3]))
#         wave = tf.Variable(lambda : tf.random.truncated_normal([10, 40]))


#         op = tf.assign(w_image, wavelet_test(w_image, img))
#         with tf.Session() as sess:
#             sess.run(tf.global_variables_initializer())
#             ret = sess.run(op)
#             print(type(ret)) # <class 'numpy.ndarray'>
#             print(ret)
#         w_image = tf.zeros(shape=[4, 4, 3])
  
#         #MM_image = np.zeros((4,4,2))
#         for num in range(0, 4):
#             for ber in range(0, 4):
#                 print((img[num*2][ber*2] - img[num*2+1][ber*2] + img[num*2][ber*2+1] - img[num*2+1][ber*2+1]))
#                 w_image[num][ber][0].assign((img[num*2][ber*2] - img[num*2+1][ber*2] + img[num*2][ber*2+1] - img[num*2+1][ber*2+1]) / 4)
#                 w_image[num][ber][1].assign((img[num*2][ber*2] + img[num*2+1][ber*2] - img[num*2][ber*2+1] - img[num*2+1][ber*2+1]) / 4)
#                 w_image[num][ber][2].assign((img[num*2][ber*2] - img[num*2+1][ber*2] - img[num*2][ber*2+1] + img[num*2+1][ber*2+1]) / 4)
                #MM_image[num][ber][0] = max([w_image[num][ber][0],w_image[num][ber][1],w_image[num][ber][2]])
                #MM_image[num][ber][1] = min([w_image[num][ber][0],w_image[num][ber][1],w_image[num][ber][2]])
            #w_images.append(MM_image)
            #w_images.append(w_image)
            
        a = img[0::2,0::2]
        b = img[0::2,1::2]
        c = img[1::2,0::2]
        d = img[1::2,1::2]
        
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print(a)
        
        test = np.zeros((8,8))
        
        #print(img.numpy())
        
        LL = (a + b + c + d) / 4
        LH = (a - b + c - d) / 4
        HL = (a + b - c - d) / 4
        HH = (a - b - c + d) / 4
        
#          wave[:][:][0].assign(LL)
#          wave[:][:][1] = LH
#          wave[:][:][2] = HL
#          wave[:][:][3] = HH
#         wave(:,:,0) = LL
#         wave(:,:,1) = LH
#         wave(:,:,2) = HL
#         wave(:,:,3) = HH
        
        #print(wave)
        #images = np.array(images)

        #if ret_wavelet:
        #return np.array(w_image)
        return wave
        #else:
            #return np.array(images), np.array(labels)
        


In [ ]:
if __name__ == '__main__':
   
    dcgan = DCGAN()
    
    r, c = 5, 5
    check_noise = np.random.uniform(-1, 1, (r * c, 100))
    dcgan.train(iterations=200000, batch_size=32, save_interval=1000,
                model_interval=5000, check_noise=check_noise, r=r,c=c)


Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_65 (Conv2D)           (None, 8, 8, 48)          240       
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 8, 8, 128)         24704     
_________________________________________________________________
dropout_26 (Dropout)         (None, 8, 8, 128)         0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 8192)              0         
_________________________________________________________________
dense_39 (Dense)             (None, 128)               1048704   
_________________________________________________________________
dropout_27 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 1)               

69 [D loss: 0.444616, acc.: 78.12%] [G loss: 1.778976]
gen_predict imgs <class 'numpy.ndarray'>
70 [D loss: 0.451528, acc.: 87.50%] [G loss: 1.840690]
gen_predict imgs <class 'numpy.ndarray'>
71 [D loss: 0.338671, acc.: 84.38%] [G loss: 1.973762]
gen_predict imgs <class 'numpy.ndarray'>
72 [D loss: 0.398747, acc.: 71.88%] [G loss: 1.892781]
gen_predict imgs <class 'numpy.ndarray'>
73 [D loss: 0.418415, acc.: 81.25%] [G loss: 1.945596]
gen_predict imgs <class 'numpy.ndarray'>
74 [D loss: 0.355112, acc.: 87.50%] [G loss: 1.950018]
gen_predict imgs <class 'numpy.ndarray'>
75 [D loss: 0.365232, acc.: 84.38%] [G loss: 1.827002]
gen_predict imgs <class 'numpy.ndarray'>
76 [D loss: 0.482969, acc.: 81.25%] [G loss: 1.648042]
gen_predict imgs <class 'numpy.ndarray'>
77 [D loss: 0.378459, acc.: 84.38%] [G loss: 1.736098]
gen_predict imgs <class 'numpy.ndarray'>
78 [D loss: 0.352545, acc.: 78.12%] [G loss: 1.882667]
gen_predict imgs <class 'numpy.ndarray'>
79 [D loss: 0.466437, acc.: 78.12%] [G l

155 [D loss: 0.663311, acc.: 59.38%] [G loss: 0.722821]
gen_predict imgs <class 'numpy.ndarray'>
156 [D loss: 0.727508, acc.: 62.50%] [G loss: 0.684113]
gen_predict imgs <class 'numpy.ndarray'>
157 [D loss: 0.716032, acc.: 50.00%] [G loss: 0.681542]
gen_predict imgs <class 'numpy.ndarray'>
158 [D loss: 0.682273, acc.: 53.12%] [G loss: 0.747925]
gen_predict imgs <class 'numpy.ndarray'>
159 [D loss: 0.737637, acc.: 59.38%] [G loss: 0.675631]
gen_predict imgs <class 'numpy.ndarray'>
160 [D loss: 0.974072, acc.: 40.62%] [G loss: 0.744195]
gen_predict imgs <class 'numpy.ndarray'>
161 [D loss: 0.694573, acc.: 50.00%] [G loss: 0.726338]
gen_predict imgs <class 'numpy.ndarray'>
162 [D loss: 0.700035, acc.: 65.62%] [G loss: 0.754945]
gen_predict imgs <class 'numpy.ndarray'>
163 [D loss: 0.709536, acc.: 50.00%] [G loss: 0.767739]
gen_predict imgs <class 'numpy.ndarray'>
164 [D loss: 0.808033, acc.: 34.38%] [G loss: 0.886891]
gen_predict imgs <class 'numpy.ndarray'>
165 [D loss: 0.653595, acc.: 5

gen_predict imgs <class 'numpy.ndarray'>
240 [D loss: 0.693297, acc.: 56.25%] [G loss: 0.701080]
gen_predict imgs <class 'numpy.ndarray'>
241 [D loss: 0.667593, acc.: 62.50%] [G loss: 0.680744]
gen_predict imgs <class 'numpy.ndarray'>
242 [D loss: 0.696498, acc.: 46.88%] [G loss: 0.688989]
gen_predict imgs <class 'numpy.ndarray'>
243 [D loss: 0.703855, acc.: 40.62%] [G loss: 0.679468]
gen_predict imgs <class 'numpy.ndarray'>
244 [D loss: 0.722830, acc.: 34.38%] [G loss: 0.685866]
gen_predict imgs <class 'numpy.ndarray'>
245 [D loss: 0.695220, acc.: 59.38%] [G loss: 0.671014]
gen_predict imgs <class 'numpy.ndarray'>
246 [D loss: 0.718515, acc.: 43.75%] [G loss: 0.671021]
gen_predict imgs <class 'numpy.ndarray'>
247 [D loss: 0.720695, acc.: 34.38%] [G loss: 0.684461]
gen_predict imgs <class 'numpy.ndarray'>
248 [D loss: 0.723416, acc.: 31.25%] [G loss: 0.699814]
gen_predict imgs <class 'numpy.ndarray'>
249 [D loss: 0.671036, acc.: 62.50%] [G loss: 0.690052]
gen_predict imgs <class 'numpy

gen_predict imgs <class 'numpy.ndarray'>
327 [D loss: 0.695304, acc.: 53.12%] [G loss: 0.721122]
gen_predict imgs <class 'numpy.ndarray'>
328 [D loss: 0.696555, acc.: 50.00%] [G loss: 0.716546]
gen_predict imgs <class 'numpy.ndarray'>
329 [D loss: 0.687279, acc.: 59.38%] [G loss: 0.717009]
gen_predict imgs <class 'numpy.ndarray'>
330 [D loss: 0.685186, acc.: 53.12%] [G loss: 0.727712]
gen_predict imgs <class 'numpy.ndarray'>
331 [D loss: 0.702900, acc.: 53.12%] [G loss: 0.722581]
gen_predict imgs <class 'numpy.ndarray'>
332 [D loss: 0.692625, acc.: 50.00%] [G loss: 0.734285]
gen_predict imgs <class 'numpy.ndarray'>
333 [D loss: 0.687149, acc.: 53.12%] [G loss: 0.715535]
gen_predict imgs <class 'numpy.ndarray'>
334 [D loss: 0.722967, acc.: 34.38%] [G loss: 0.725921]
gen_predict imgs <class 'numpy.ndarray'>
335 [D loss: 0.712399, acc.: 40.62%] [G loss: 0.728850]
gen_predict imgs <class 'numpy.ndarray'>
336 [D loss: 0.707543, acc.: 37.50%] [G loss: 0.717627]
gen_predict imgs <class 'numpy

...

9780 [D loss: 0.671897, acc.: 56.25%] [G loss: 0.696778]
gen_predict imgs <class 'numpy.ndarray'>
9781 [D loss: 0.685990, acc.: 62.50%] [G loss: 0.696521]
gen_predict imgs <class 'numpy.ndarray'>
9782 [D loss: 0.671519, acc.: 53.12%] [G loss: 0.697091]
gen_predict imgs <class 'numpy.ndarray'>
9783 [D loss: 0.694645, acc.: 40.62%] [G loss: 0.698485]
gen_predict imgs <class 'numpy.ndarray'>
9784 [D loss: 0.695547, acc.: 43.75%] [G loss: 0.697267]
gen_predict imgs <class 'numpy.ndarray'>
9785 [D loss: 0.686818, acc.: 56.25%] [G loss: 0.699303]
gen_predict imgs <class 'numpy.ndarray'>
9786 [D loss: 0.651032, acc.: 50.00%] [G loss: 0.693464]
gen_predict imgs <class 'numpy.ndarray'>
9787 [D loss: 0.695067, acc.: 46.88%] [G loss: 0.699939]
gen_predict imgs <class 'numpy.ndarray'>
9788 [D loss: 0.692667, acc.: 40.62%] [G loss: 0.697833]
gen_predict imgs <class 'numpy.ndarray'>
9789 [D loss: 0.671051, acc.: 59.38%] [G loss: 0.695720]
gen_predict imgs <class 'numpy.ndarray'>
9790 [D loss: 0.6931

9864 [D loss: 0.695449, acc.: 53.12%] [G loss: 0.687593]
gen_predict imgs <class 'numpy.ndarray'>
9865 [D loss: 0.788664, acc.: 46.88%] [G loss: 0.725438]
gen_predict imgs <class 'numpy.ndarray'>
9866 [D loss: 0.686560, acc.: 53.12%] [G loss: 0.723923]
gen_predict imgs <class 'numpy.ndarray'>
9867 [D loss: 0.677111, acc.: 43.75%] [G loss: 0.779999]
gen_predict imgs <class 'numpy.ndarray'>
9868 [D loss: 0.659081, acc.: 53.12%] [G loss: 0.721462]
gen_predict imgs <class 'numpy.ndarray'>
9869 [D loss: 0.667679, acc.: 46.88%] [G loss: 0.719751]
gen_predict imgs <class 'numpy.ndarray'>
9870 [D loss: 0.672416, acc.: 59.38%] [G loss: 0.720118]
gen_predict imgs <class 'numpy.ndarray'>
9871 [D loss: 0.646912, acc.: 53.12%] [G loss: 0.744911]
gen_predict imgs <class 'numpy.ndarray'>
9872 [D loss: 0.692008, acc.: 46.88%] [G loss: 0.720988]
gen_predict imgs <class 'numpy.ndarray'>
9873 [D loss: 0.675963, acc.: 56.25%] [G loss: 0.700242]
gen_predict imgs <class 'numpy.ndarray'>
9874 [D loss: 0.6981

gen_predict imgs <class 'numpy.ndarray'>
9949 [D loss: 0.693971, acc.: 50.00%] [G loss: 0.702756]
gen_predict imgs <class 'numpy.ndarray'>
9950 [D loss: 0.670544, acc.: 53.12%] [G loss: 0.701186]
gen_predict imgs <class 'numpy.ndarray'>
9951 [D loss: 0.693460, acc.: 59.38%] [G loss: 0.702427]
gen_predict imgs <class 'numpy.ndarray'>
9952 [D loss: 0.693223, acc.: 50.00%] [G loss: 0.702232]
gen_predict imgs <class 'numpy.ndarray'>
9953 [D loss: 0.692983, acc.: 59.38%] [G loss: 0.700161]
gen_predict imgs <class 'numpy.ndarray'>
9954 [D loss: 0.690440, acc.: 56.25%] [G loss: 0.697625]
gen_predict imgs <class 'numpy.ndarray'>
9955 [D loss: 0.691776, acc.: 56.25%] [G loss: 0.699309]
gen_predict imgs <class 'numpy.ndarray'>
9956 [D loss: 0.674747, acc.: 34.38%] [G loss: 0.699035]
gen_predict imgs <class 'numpy.ndarray'>
9957 [D loss: 0.671094, acc.: 56.25%] [G loss: 0.701508]
gen_predict imgs <class 'numpy.ndarray'>
9958 [D loss: 0.694007, acc.: 50.00%] [G loss: 0.700437]
gen_predict imgs <cl

gen_predict imgs <class 'numpy.ndarray'>
10032 [D loss: 0.694043, acc.: 50.00%] [G loss: 0.706097]
gen_predict imgs <class 'numpy.ndarray'>
10033 [D loss: 0.691739, acc.: 53.12%] [G loss: 0.704659]
gen_predict imgs <class 'numpy.ndarray'>
10034 [D loss: 0.672029, acc.: 53.12%] [G loss: 0.704743]
gen_predict imgs <class 'numpy.ndarray'>
10035 [D loss: 0.647460, acc.: 56.25%] [G loss: 0.687178]
gen_predict imgs <class 'numpy.ndarray'>
10036 [D loss: 0.693838, acc.: 50.00%] [G loss: 0.702525]
gen_predict imgs <class 'numpy.ndarray'>
10037 [D loss: 0.692681, acc.: 53.12%] [G loss: 0.705069]
gen_predict imgs <class 'numpy.ndarray'>
10038 [D loss: 0.750089, acc.: 46.88%] [G loss: 0.705670]
gen_predict imgs <class 'numpy.ndarray'>
10039 [D loss: 0.685566, acc.: 50.00%] [G loss: 0.705236]
gen_predict imgs <class 'numpy.ndarray'>
10040 [D loss: 0.673468, acc.: 50.00%] [G loss: 0.704224]
gen_predict imgs <class 'numpy.ndarray'>
10041 [D loss: 0.692158, acc.: 50.00%] [G loss: 0.702987]
gen_predic

gen_predict imgs <class 'numpy.ndarray'>
10115 [D loss: 0.692141, acc.: 53.12%] [G loss: 0.699772]
gen_predict imgs <class 'numpy.ndarray'>
10116 [D loss: 0.690702, acc.: 65.62%] [G loss: 0.700162]
gen_predict imgs <class 'numpy.ndarray'>
10117 [D loss: 0.692585, acc.: 56.25%] [G loss: 0.702051]
gen_predict imgs <class 'numpy.ndarray'>
10118 [D loss: 0.672833, acc.: 53.12%] [G loss: 0.684179]
gen_predict imgs <class 'numpy.ndarray'>
10119 [D loss: 0.690843, acc.: 59.38%] [G loss: 0.695956]
gen_predict imgs <class 'numpy.ndarray'>
10120 [D loss: 0.692655, acc.: 53.12%] [G loss: 0.676888]
gen_predict imgs <class 'numpy.ndarray'>
10121 [D loss: 0.689419, acc.: 56.25%] [G loss: 0.699574]
gen_predict imgs <class 'numpy.ndarray'>
10122 [D loss: 0.670937, acc.: 56.25%] [G loss: 0.697906]
gen_predict imgs <class 'numpy.ndarray'>
10123 [D loss: 0.692689, acc.: 53.12%] [G loss: 0.699188]
gen_predict imgs <class 'numpy.ndarray'>
10124 [D loss: 0.690573, acc.: 59.38%] [G loss: 0.701335]
gen_predic

In [ ]:
import tensorflow as tf

if __name__ == '__main__':
    
    a = np.array([[1, 2]])
    b = np.array([1, 2])
    
    
    print(a.shape, b.shape)
    

    const1 = tf.constant(2)
    const2 = tf.constant(3, dtype=tf.float32)
    add_op = tf.add(const1, const2) # 2 + 3
    w = tf.zeros([3, 2])
    
#     with tf.Session() as sess:
#         result = sess.run(tf.add(w, const2))
#         print(result)
        
        

In [77]:
def func(x):
    a = np.zeros(x.shape)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            a[i,j] = 10*i+j
    return a

def func2(x, adder):
    a = np.zeros(x.shape)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            a[i,j] = 10*i+j + adder

def func3(x, adder):
    a = np.zeros(x.shape)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            a[i,j] = 10*i+j + adder
    return a

In [6]:
# params x: (?, ?, 3)
def wavelet_test(x, img, w, h):
    x = np.zeros(x.shape)
    
    h, w, _ = img.shape
    print(h, w)
#     for num in range(0, w//2):
#         for ber in range(0, h//2):
# #             print((img[num*2][ber*2] - img[num*2+1][ber*2] + img[num*2][ber*2+1] - img[num*2+1][ber*2+1]))
#             x[num][ber][0].assign((img[num*2][ber*2] - img[num*2+1][ber*2] + img[num*2][ber*2+1] - img[num*2+1][ber*2+1]) / 4)
#             x[num][ber][1].assign((img[num*2][ber*2] + img[num*2+1][ber*2] - img[num*2][ber*2+1] - img[num*2+1][ber*2+1]) / 4)
#             x[num][ber][2].assign((img[num*2][ber*2] - img[num*2+1][ber*2] - img[num*2][ber*2+1] + img[num*2+1][ber*2+1]) / 4)
    return x

In [7]:
# params x: (?, ?, 3)
def wavelet_internal(x, img, w, h):
    x = np.zeros(x.shape)
    
    for num in range(0, w//2):
        for ber in range(0, h//2):
#             print((img[num*2][ber*2] - img[num*2+1][ber*2] + img[num*2][ber*2+1] - img[num*2+1][ber*2+1]))
            x[num][ber][0].assign((img[num*2][ber*2] - img[num*2+1][ber*2] + img[num*2][ber*2+1] - img[num*2+1][ber*2+1]) / 4)
            x[num][ber][1].assign((img[num*2][ber*2] + img[num*2+1][ber*2] - img[num*2][ber*2+1] - img[num*2+1][ber*2+1]) / 4)
            x[num][ber][2].assign((img[num*2][ber*2] - img[num*2+1][ber*2] - img[num*2][ber*2+1] + img[num*2+1][ber*2+1]) / 4)
    return x

In [5]:
import copy
x = tf.Variable(lambda: tf.zeros((4,3)))
print(x)
y = copy.deepcopy(x)
tf.compat.v1.assign(x, x+2)
print(x)

# with tf.compat.v1.Session() as sess:
#     sess.run(tf.compat.v1.global_variables_initializer())
#     ret = sess.run(op)
#     print(type(ret)) # <class 'numpy.ndarray'>
#     print(ret)

<tf.Variable 'Variable:0' shape=(4, 3) dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(4, 3) dtype=float32, numpy=
array([[2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.]], dtype=float32)>


In [65]:
c = tf.constant([[1.0, 2.0], [3.0, 4.0]])
d = tf.constant([[1.0, 1.0], [0.0, 1.0]])
e = tf.matmul(c, d)
print(e)
print(e.np())

Tensor("MatMul_2:0", shape=(2, 2), dtype=float32)


AttributeError: 'Tensor' object has no attribute 'np'

In [68]:
add = lambda a, b=1 : a + b

In [4]:
tf.executing_eagerly()

True

In [3]:
tf.compat.v1.enable_eager_execution()

In [25]:
!pip install np_utils

     |████████████████████████████████| 61 kB 2.3 MB/s eta 0:00:011
     |████████████████████████████████| 829 kB 4.8 MB/s eta 0:00:01
  Created wheel for np-utils: filename=np_utils-0.5.12.1-py3-none-any.whl size=57125 sha256=06602e4a8a83b3488c97ba5295c970ab5f026e169679754c90bb4748b1971756
  Stored in directory: /root/.cache/pip/wheels/58/98/54/2896a40fd91932a8a2568e688f87231f7da2eaad330254335a
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=8488ae5cb039e707431b78447f48019dd7ea3bd3b60f8c24ef0e72d538cd0614
  Stored in directory: /root/.cache/pip/wheels/6e/9c/ed/4499c9865ac1002697793e0ae05ba6be33553d098f3347fb94
Successfully built np-utils future
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [6]:
!pwd

/tmp/py/in_report
